In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [23]:
testSites = pd.read_csv('./data_sets/NC_testSites.csv')
byRace = pd.read_csv('./data_sets/NC_counties_byRace.csv')

/Users/jincho/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning:

Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.



In [24]:
testSites_cond = testSites.drop(columns=['municipality','Unnamed: 0', 'name', 'agency', 'phone', 'agencyurl', 'Instructions', 'drive_through', 'appt_only', 'call_first', 'health_dept_url', 'State'])
testSites_cond = testSites_cond[(testSites_cond.status!='Closed') & (testSites_cond.status!='Temporarily Closed')]

In [25]:
byRace['TOT_WA'] = byRace['WA_MALE'] + byRace['WA_FEMALE']
byRace['TOT_BA'] = byRace['BA_MALE'] + byRace['BA_FEMALE']
byRace['TOT_IA'] = byRace['IA_MALE'] + byRace['IA_FEMALE']
byRace['TOT_AA'] = byRace['AA_MALE'] + byRace['AA_FEMALE']
byRace['TOT_NA'] = byRace['NA_MALE'] + byRace['NA_FEMALE']
byRace['TOT_TOM'] = byRace['TOM_MALE'] + byRace['TOM_FEMALE']


In [37]:
comb_test_ba = pd.DataFrame(columns = ['county', 'testSites', 'totPop', \
                                         'totBA', 'totNBA', 'totWA', 'totIA', 'totAA', 'totNA', 'totTOM'])
comb_test_ba.head()

,county,testSites,totPop,totBA,totNBA,totWA,totIA,totAA,totNA,totTOM


In [38]:
for county in testSites_cond.county.unique():
    if(county in byRace.CTYNAME.unique()):
        num_testSites = len(testSites_cond[testSites_cond.county==county])
        totPop = byRace[byRace['CTYNAME']==county].TOT_POP.iloc[0]
        totBA = byRace[byRace['CTYNAME']==county].TOT_BA.iloc[0]
        totNBA = totPop - totBA
        totWA = byRace[byRace['CTYNAME']==county].TOT_WA.iloc[0]
        totIA = byRace[byRace['CTYNAME']==county].TOT_IA.iloc[0]
        totAA = byRace[byRace['CTYNAME']==county].TOT_AA.iloc[0]
        totNA = byRace[byRace['CTYNAME']==county].TOT_NA.iloc[0]
        totTOM = byRace[byRace['CTYNAME']==county].TOT_TOM.iloc[0]
        comb_test_ba = comb_test_ba.append({'county':county, 'testSites':num_testSites, 'totPop':totPop, \
                                                'totBA':totBA, 'totNBA':totNBA, 'totWA':totWA, 'totIA':totIA,\
                                           'totAA':totAA, 'totNA':totNA, 'totTOM':totTOM},ignore_index=True)

In [39]:
comb_test_ba.head()

,county,testSites,totPop,totBA,totNBA,totWA,totIA,totAA,totNA,totTOM
0,Forsyth,20,354524,96266,258258,240960,2899,7209,386,6804
1,Guilford,23,494953,164981,329972,295333,3515,20682,366,10076
2,Alamance,5,152692,29247,123445,116436,2111,2013,101,2784
3,Wake,49,928787,197743,731044,649405,7712,52873,598,20456
4,Moore,5,89262,11983,77279,74028,805,866,143,1437


In [29]:
comb_test_ba = comb_test_ba.set_index('county')

In [57]:
comb_test_ba["BA_to_sites"] = comb_test_ba['totBA'] / comb_test_ba['testSites']
comb_test_ba["NBA_to_sites"] = comb_test_ba['totNBA'] / comb_test_ba['testSites']
comb_test_ba["WA_to_sites"] = comb_test_ba['totWA'] / comb_test_ba['testSites']
comb_test_ba["IA_to_sites"] = comb_test_ba['totIA'] / comb_test_ba['testSites']
comb_test_ba["AA_to_sites"] = comb_test_ba['totAA'] / comb_test_ba['testSites']
comb_test_ba["NA_to_sites"] = comb_test_ba['totNA'] / comb_test_ba['testSites']
comb_test_ba["TOM_to_sites"] = comb_test_ba['totTOM'] / comb_test_ba['testSites']

In [41]:
comb_test_ba["testSites"] = comb_test_ba["testSites"].astype(float)
comb_test_ba["totPop"] = comb_test_ba["totPop"].astype(float)
comb_test_ba["totBA"] = comb_test_ba["totBA"].astype(float)
comb_test_ba["totNBA"] = comb_test_ba["totNBA"].astype(float)
comb_test_ba["totWA"] = comb_test_ba["totWA"].astype(float)
comb_test_ba["totIA"] = comb_test_ba["totIA"].astype(float)
comb_test_ba["totAA"] = comb_test_ba["totAA"].astype(float)
comb_test_ba["totNA"] = comb_test_ba["totNA"].astype(float)
comb_test_ba["totTOM"] = comb_test_ba["totTOM"].astype(float)

In [54]:
np.array(comb_test_ba[comb_test_ba.testSites>=10].county)


array(['Forsyth', 'Guilford', 'Wake', 'Durham', 'Onslow', 'New Hanover',
       'Buncombe', 'Mecklenburg', 'Cumberland'], dtype=object)

In [59]:
counties = np.array(comb_test_ba[comb_test_ba.testSites>=7].sort_values(by='testSites').county)
BA = np.array(comb_test_ba[comb_test_ba.testSites>=7].sort_values(by='testSites').BA_to_sites)
WA = np.array(comb_test_ba[comb_test_ba.testSites>=7].sort_values(by='testSites').WA_to_sites)
AA = np.array(comb_test_ba[comb_test_ba.testSites>=7].sort_values(by='testSites').AA_to_sites)
TOM = np.array(comb_test_ba[comb_test_ba.testSites>=7].sort_values(by='testSites').TOM_to_sites)
fig = go.Figure(data=[
    go.Bar(name='Black Pop', x=counties, y=BA),
    go.Bar(name='White Pop', x=counties, y=WA),
    go.Bar(name='AA Pop', x=counties, y=AA),
    go.Bar(name='TOM Pop', x=counties, y=TOM),
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()

In [73]:
comb_test_ba["BA_perc"] = np.round(((comb_test_ba['totBA'] / comb_test_ba['totPop'])*100),3)
comb_test_ba["NBA_perc"] = np.round(((comb_test_ba['totNBA'] / comb_test_ba['totPop'])*100),3)
comb_test_ba["WA_perc"] = np.round(((comb_test_ba['totWA'] / comb_test_ba['totPop'])*100),3)
comb_test_ba["IA_perc"] = np.round(((comb_test_ba['totIA'] / comb_test_ba['totPop'])*100),3)
comb_test_ba["AA_perc"] = np.round(((comb_test_ba['totAA'] / comb_test_ba['totPop'])*100),3)
comb_test_ba["NA_perc"] = np.round(((comb_test_ba['totNA'] / comb_test_ba['totPop'])*100),3)
comb_test_ba["TOM_perc"] = np.round(((comb_test_ba['totTOM'] / comb_test_ba['totPop'])*100),3)

In [64]:
counties = np.array(comb_test_ba[comb_test_ba.testSites>=5].sort_values(by='testSites').county)
BA = np.array(comb_test_ba[comb_test_ba.testSites>=5].sort_values(by='testSites').BA_perc)
WA = np.array(comb_test_ba[comb_test_ba.testSites>=5].sort_values(by='testSites').WA_perc)
AA = np.array(comb_test_ba[comb_test_ba.testSites>=5].sort_values(by='testSites').AA_perc)
TOM = np.array(comb_test_ba[comb_test_ba.testSites>=5].sort_values(by='testSites').TOM_perc)
fig = go.Figure(data=[
    go.Bar(name='Black Pop', x=counties, y=BA),
    go.Bar(name='White Pop', x=counties, y=WA),
    go.Bar(name='AA Pop', x=counties, y=AA),
    go.Bar(name='TOM Pop', x=counties, y=TOM),
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()

In [66]:
counties = np.array(comb_test_ba[comb_test_ba.testSites<5].sort_values(by='testSites').county)
BA = np.array(comb_test_ba[comb_test_ba.testSites<5].sort_values(by='testSites').BA_perc)
WA = np.array(comb_test_ba[comb_test_ba.testSites<5].sort_values(by='testSites').WA_perc)
AA = np.array(comb_test_ba[comb_test_ba.testSites<5].sort_values(by='testSites').AA_perc)
TOM = np.array(comb_test_ba[comb_test_ba.testSites<5].sort_values(by='testSites').TOM_perc)
fig = go.Figure(data=[
    go.Bar(name='Black Pop', x=counties, y=BA),
    go.Bar(name='White Pop', x=counties, y=WA),
    go.Bar(name='AA Pop', x=counties, y=AA),
    go.Bar(name='TOM Pop', x=counties, y=TOM),
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()

In [71]:
counties = comb_test_ba.sort_values(by='testSites').county
BA = comb_test_ba.sort_values(by='testSites').BA_perc
WA = comb_test_ba.sort_values(by='testSites').WA_perc
fig = go.Figure(data=[
    go.Bar(name='Black Pop', x=counties, y=BA),
    go.Bar(name='White Pop', x=counties, y=WA)
#     go.Bar(name='AA Pop', x=counties, y=AA),
#     go.Bar(name='TOM Pop', x=counties, y=TOM),
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()

In [77]:
comb_test_ba["WA_to_BA"] = np.round((comb_test_ba['totWA'] / comb_test_ba['totBA']),3)
comb_test_ba["WA_to_min"] = np.round((comb_test_ba['totWA'] / (comb_test_ba['totPop']-comb_test_ba['totWA'])),3)

# comb_test_ba["TOM_perc"] = np.round((comb_test_ba['totTOM'] / comb_test_ba['totPop']),3)

In [79]:
counties = np.array(comb_test_ba[comb_test_ba.testSites>=5].sort_values(by='testSites').county)
WA_to_BA = np.array(comb_test_ba[comb_test_ba.testSites>=5].sort_values(by='testSites').WA_to_min)
# WA = comb_test_ba.sort_values(by='testSites').WA_perc
fig = go.Figure(data=[
    go.Bar(name='Black Pop', x=counties, y=WA_to_BA)
#     go.Bar(name='White Pop', x=counties, y=WA)
#     go.Bar(name='AA Pop', x=counties, y=AA),
#     go.Bar(name='TOM Pop', x=counties, y=TOM),
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()